# MFE230T

## Jen-Chieh Cheng

In [1]:
%matplotlib inline

import json
import gensim
import nltk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pprint
import re
import string
import time

import os

from collections import defaultdict
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string

Using TensorFlow backend.


### Selecting signals from each tweet

In [2]:
tweets_file = open("./tweets/1502820001-tweets.txt", 'rb')
lines = tweets_file.readlines()
print "Number of tweets: %d" % len(lines)
tweets_file.close()

Number of tweets: 330


In [3]:
pprint.pprint(json.loads(lines[0].strip()))

{u'contributors': None,
 u'coordinates': None,
 u'created_at': u'Tue Aug 15 17:58:26 +0000 2017',
 u'entities': {u'hashtags': [{u'indices': [45, 52], u'text': u'patent'},
                             {u'indices': [65, 68], u'text': u'IP'}],
               u'symbols': [{u'indices': [103, 108], u'text': u'GOOG'},
                            {u'indices': [109, 112], u'text': u'FB'}],
               u'urls': [{u'display_url': u'iam-media.com/blog/Detail.as\u2026',
                          u'expanded_url': u'http://www.iam-media.com/blog/Detail.aspx?g=afc6cc58-706a-475d-906a-fd85bd1e49f1',
                          u'indices': [113, 136],
                          u'url': u'https://t.co/FiHWRiETq3'}],
               u'user_mentions': [{u'id': 108564136,
                                   u'id_str': u'108564136',
                                   u'indices': [3, 16],
                                   u'name': u'IAM',
                                   u'screen_name': u'IAM_magazine'}]},
 

In [4]:
# all tweets files

lines = list()

for f in os.listdir('tweets'):
    print f
    tweet_file = open('tweets/' + f, 'rb')
    lines += tweet_file.readlines()
    tweet_file.close()
    
len(lines)

1502820001-tweets.txt
1502906402-tweets.txt
1502992801-tweets.txt
1503079201-tweets.txt
1503165601-tweets.txt
1503252001-tweets.txt
1503338401-tweets.txt
1503424801-tweets.txt
1503511201-tweets.txt
1503597601-tweets.txt
1503684001-tweets.txt
1503770401-tweets.txt
1503856801-tweets.txt
1503943201-tweets.txt
1504029601-tweets.txt
1504116001-tweets.txt
1504202401-tweets.txt
1504288801-tweets.txt
1504375201-tweets.txt
1504461601-tweets.txt
1504548001-tweets.txt
1504634401-tweets.txt
1504720801-tweets.txt
1504807201-tweets.txt
1504893601-tweets.txt
1504980001-tweets.txt
1505066401-tweets.txt
1505152801-tweets.txt
1505239201-tweets.txt
1505325601-tweets.txt
1505412001-tweets.txt
1505498401-tweets.txt
1505584801-tweets.txt
1505671201-tweets.txt
1505757601-tweets.txt
1505844001-tweets.txt
1505930401-tweets.txt


15513

### Extracting features

In [5]:
#TODO use all twitter files
data = defaultdict(dict)
i=0

for line in lines:

    tweet = json.loads(line.strip())
    if 'text' in tweet: # only messages contains 'text' field is a tweet
        ts = time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')
        data[i]["time"] = time.mktime(ts)  
        data[i]["text"] = tweet['text']
    if 'urls' in tweet['entities']:
        #print tweet['entities']['urls']
        data[i]["urls"] = len(tweet['entities']['urls'])
    if 'media' in tweet['entities']:
        #print tweet['entities']['media']
        data[i]["media"] = len(tweet['entities']['media'])
    if 'hashtags' in tweet['entities']:
        data[i]["hashtags"] = len(tweet['entities']['hashtags'])
    i += 1


In [6]:
### which other signals could be useful? 
print data[0]
print data[1]

{'text': u'RT @IAM_magazine: Exclusive: In major Valley #patent move Google #IP head Allen Lo is joining Facebook $GOOG $FB https://t.co/FiHWRiETq3', 'hashtags': 2, 'urls': 1, 'time': 1502845106.0}
{'text': u'RT @arnabch01: #investors massive bubble in #tech be careful $AAPL $GOOG $MSFT $AMZN $FB $NFLX $TSLA $CSCO $INTC $NVDA $ZNGA $ORCL $JD $MU\u2026', 'hashtags': 2, 'urls': 0, 'time': 1502844958.0}


In [7]:
#working with text
#tokenizer for tweets
tknzr = TweetTokenizer(strip_handles=True) #(strip_handles=True, reduce_len=True)

for i, info in data.items():  
    text = info['text'].lower().encode('utf-8').decode('ascii','ignore') # content of the tweet
    #print text
    text = re.sub(r"http\S*", '', text) #remove urls
    text = re.sub(r"^rt", '', text) #remove rt
    #print text
    words = tknzr.tokenize(text)
    #print words
    data[i]['exclamations'] = words.count('!')
    data[i]['questions'] = words.count('?')
    data[i]['dollar'] = words.count('$')
    data[i]['text'] = " ".join(words).encode('utf-8')
    data[i]['text'] = re.sub(r'[^\w\s]', '', data[i]['text'])
    data[i]['text'] = re.sub(r'\d+', '', data[i]['text'])
    data[i]['num_words'] = len(text) 
          

In [8]:
print data[0]
print data[1]

{'urls': 1, 'text': ' exclusive  in major valley patent move google ip head allen lo is joining facebook  goog  fb', 'hashtags': 2, 'dollar': 2, 'questions': 0, 'time': 1502845106.0, 'exclamations': 0, 'num_words': 111}
{'urls': 0, 'text': ' investors massive bubble in tech be careful  aapl  goog  msft  amzn  fb  nflx  tsla  csco  intc  nvda  znga  orcl  jd  mu', 'hashtags': 2, 'dollar': 14, 'questions': 0, 'time': 1502844958.0, 'exclamations': 0, 'num_words': 136}


### Understanding the data

In [9]:
df = pd.DataFrame.from_dict(data, orient='index')
df.describe()

,urls,hashtags,dollar,questions,time,exclamations,num_words,media
count,15513.000000,15513.000000,15513.000000,15513.000000,1.551300e+04,15513.000000,15513.000000,1153.0
mean,0.684200,0.853671,3.888094,0.096629,1.504373e+09,0.115516,95.372075,1.0
std,0.536803,1.932290,4.349792,0.334164,9.147151e+05,0.450559,30.699871,0.0
min,0.000000,0.000000,0.000000,0.000000,1.502505e+09,0.000000,6.000000,1.0
25%,0.000000,0.000000,1.000000,0.000000,1.503607e+09,0.000000,72.000000,1.0
50%,1.000000,0.000000,2.000000,0.000000,1.504389e+09,0.000000,99.000000,1.0
75%,1.000000,1.000000,5.000000,0.000000,1.505175e+09,0.000000,117.000000,1.0
max,3.000000,12.000000,24.000000,6.000000,1.505956e+09,7.000000,152.000000,1.0


In [10]:
df.head()

,urls,text,hashtags,dollar,questions,time,exclamations,num_words,media
0,1,exclusive in major valley patent move google...,2,2,0,1.502845e+09,0,111,NaN
1,0,investors massive bubble in tech be careful ...,2,14,0,1.502845e+09,0,136,NaN
2,0,google goog is the embodiment of modern pc f...,6,1,0,1.502845e+09,0,126,NaN
3,0,timberr iwm spy tlt gs gld btc goog ...,0,21,0,1.502845e+09,2,137,NaN
4,1,bank of nova scotia buys shares of alphabet i...,0,1,0,1.502845e+09,0,62,NaN


### Data Cleaning & Feature Engineering

In [11]:
df = df.drop_duplicates('text')
df.shape

(8434, 9)

In [12]:
def shrink_word(w):
    
    if len(w) < 3:
        return w
    else:
        w = list(w)
        i = 2
        while i < len(w):
            if w[i-1] == w[i-2] and w[i-1] == w[i]:
                w.pop(i)
            else:
                i+=1
        return ''.join(w)
    
shrink_word('amaaaaaazing jjjjjjaayy chou')

'amaazing jjaayy chou'

In [13]:
LOWER_SPACE = string.lowercase + ' '

def text_parser(text):
    
    cleaned = text.lower().replace('\n',' ').replace('\t', ' ')
    
    for char in string.punctuation:
        cleaned = cleaned.replace(char,' ')
    cleaned = filter(lambda char: char in LOWER_SPACE, cleaned)
    
    cleaned = ' '.join(map(shrink_word, cleaned.split()))
    
    return str(cleaned)

In [14]:
text_parser(df.text[30])

KeyError: 30

In [15]:
df['text'] = df['text'].apply(text_parser)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

word_vectorizer = CountVectorizer(analyzer='word', stop_words='english')
sparse_matrix = word_vectorizer.fit_transform(df['text'])
frequencies = sum(sparse_matrix).toarray()[0]
words = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])
print words.describe()
words.head(10)

          frequency
count  11047.000000
mean       7.113334
std       42.310521
min        1.000000
25%        1.000000
50%        2.000000
75%        4.000000
max     1632.000000


,frequency
aa,23
aab,2
aaba,5
aac,1
aae,1
aal,8
aanndd,1
aaoi,7
aap,10
aapl,1632


In [17]:
word_vectorizer = CountVectorizer(analyzer='word', stop_words='english',min_df=5, max_df=3000)
sparse_matrix = word_vectorizer.fit_transform(df['text'])
frequencies = sum(sparse_matrix).toarray()[0]
words = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])
print words.describe()
words.head(10)

         frequency
count  2480.000000
mean     25.948387
std      86.694647
min       5.000000
25%       6.000000
50%      10.000000
75%      20.000000
max    1632.000000


,frequency
aa,23
aaba,5
aal,8
aaoi,7
aap,10
aapl,1632
aapls,11
ab,6
abbv,11
abeo,5


### Finding structure in text

In [18]:
del words

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

n_features = 250
n_components = 20
n_top_words = 10


### Counts
tf_vectorizer = CountVectorizer(min_df=5, max_df=3000, stop_words='english')
tf = tf_vectorizer.fit_transform(df.text)
tf_feature_names = tf_vectorizer.get_feature_names()
#print tf_feature_names

In [20]:
## TF-IDF
tfidf_vectorizer = TfidfVectorizer(min_df=5,max_df=3000,stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df.text)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

### Finding topics with LDA

**Remark**:
As we exploit all tweet files, the both LDA models (TF and TFIDF) generate more diversified topics.

In [21]:
lda = LatentDirichletAllocation(n_components=n_components, learning_method='online')
lda.fit(tf)
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0: long data way days aapl future volume technology did run
Topic #1: corporation shares business management position alphabet international ibm llc stake
Topic #2: tsla options gm hold better cars break investors bond half
Topic #3: stocks trading fb check tech twtr video nice alerts watch
Topic #4: amzn trade amazon wmt good car th tgt think really
Topic #5: snap time snapchat rating looking value soon does coming china
Topic #6: im review musk past study build program user talk ios
Topic #7: aapl fb spy tsla amzn nflx baba stocks nvda qq
Topic #8: tsla aapl tesla apple like calls year model apples profits
Topic #9: msft twtr crm ibm aapl brk agn amzn vrx iep
Topic #10: aapl short apple chart look jnj ceo september tv youre
Topic #11: box cloud sold holding buys ipo acquires home moving entry
Topic #12: msft microsoft new today blockchain ready high highs relevant event
Topic #13: big holdings news daily lol azfl profit huge drys blue
Topic #14: earnings vs intc corp analysis 

In [22]:
lda = LatentDirichletAllocation(n_components=n_components, learning_method='online')
lda.fit(tfidf)
print_top_words(lda, tfidf_feature_names, n_top_words)

Topic #0: tsla spy long nflx qq aapl car amzn fb tgt
Topic #1: business ibm international machines corporation bought sold shares bank amzn
Topic #2: snap getting rating buy great box chart ive trades day
Topic #3: holdings ibm tsla crm brk msft amzn model tesla agn
Topic #4: fb amzn googl goog snap time data earnings year billion
Topic #5: look think vs china highs finally blue ibm analysis better
Topic #6: like price just tsla days share hit make stock watson
Topic #7: good going trade support holding really snap company puts recommend
Topic #8: microsoft alphabet msft management shares position llc goog stake capital
Topic #9: aapl stocks investing stockmarket watch nvda amzn intc goog amd
Topic #10: relevant invest life premium azure launches development acquires ahgif chk
Topic #11: trading fb tsla twtr snap aapl money stocks free bitcoin
Topic #12: break play key action news musk drys update discovering monday
Topic #13: new best companies dividend high aapl moving does lnkd arti

### Simple sentiment analysis

In [23]:
positive = pd.read_csv('data/positive-words.txt', names=['a'])
positive = set(positive['a'].tolist())

negative = pd.read_csv('data/negative-words.txt', names=['a'])
negative = set(negative['a'].tolist())

In [24]:
count_positive = []
count_negative = []
for i, row in df.iterrows():
    commonp = set(row['text'].split()).intersection(positive) 
    count_positive.append(len(commonp))
    commonn = set(row['text'].split()).intersection(negative) 
    count_negative.append(len(commonn))

In [25]:
df['positive'] = count_positive
df['negative'] = count_negative
df['label'] = [int(p > 0) - int(n > 0) for p, n in zip(count_positive, count_negative)]

df.head(10)

,urls,text,hashtags,dollar,questions,time,exclamations,num_words,media,positive,negative,label
0,1,exclusive in major valley patent move google i...,2,2,0,1.502845e+09,0,111,NaN,0,0,0
1,0,investors massive bubble in tech be careful aa...,2,14,0,1.502845e+09,0,136,NaN,0,0,0
2,0,google goog is the embodiment of modern pc fas...,6,1,0,1.502845e+09,0,126,NaN,2,1,0
3,0,timberr iwm spy tlt gs gld btc goog aapl fb nf...,0,21,0,1.502845e+09,2,137,NaN,0,0,0
4,1,bank of nova scotia buys shares of alphabet in...,0,1,0,1.502845e+09,0,62,NaN,0,0,0
5,1,alphabet inc goog stake raised by north star a...,0,1,0,1.502845e+09,0,69,NaN,0,0,0
6,1,the machines keep getting smarter can your por...,0,2,1,1.502845e+09,0,102,NaN,2,0,1
8,0,as alphabet goog valuation rose robshaw julian...,0,2,0,1.502844e+09,0,105,NaN,0,0,0
9,1,warren averett asset management llc boosts pos...,0,1,0,1.502844e+09,0,75,NaN,0,0,0
10,1,goog himx vuzi great article,0,3,0,1.502844e+09,0,37,NaN,1,0,1


In [26]:
df.label.value_counts()

 0    4969
 1    2219
-1    1246
Name: label, dtype: int64

In [27]:
df.describe()

,urls,hashtags,dollar,questions,time,exclamations,num_words,media,positive,negative,label
count,8434.000000,8434.000000,8434.000000,8434.000000,8.434000e+03,8434.000000,8434.000000,792.0,8434.000000,8434.000000,8434.000000
mean,0.701209,0.521224,2.894475,0.107897,1.504405e+09,0.117501,90.078018,1.0,0.424354,0.277330,0.115366
std,0.541971,1.298674,3.413585,0.358498,9.296125e+05,0.460790,30.275436,0.0,0.665263,0.567962,0.630535
min,0.000000,0.000000,0.000000,0.000000,1.502506e+09,0.000000,6.000000,1.0,0.000000,0.000000,-1.000000
25%,0.000000,0.000000,1.000000,0.000000,1.503618e+09,0.000000,68.000000,1.0,0.000000,0.000000,0.000000
50%,1.000000,0.000000,2.000000,0.000000,1.504391e+09,0.000000,92.000000,1.0,0.000000,0.000000,0.000000
75%,1.000000,0.000000,3.000000,0.000000,1.505261e+09,0.000000,114.000000,1.0,1.000000,0.000000,1.000000
max,3.000000,12.000000,24.000000,6.000000,1.505956e+09,7.000000,152.000000,1.0,6.000000,4.000000,1.000000


### Positive sentiment

In [28]:
df[(df['positive'] >0) & (df['negative']  == 0)]['text'].head(50)

6      the machines keep getting smarter can your por...
10                          goog himx vuzi great article
11     apples bargaining power rising google said to ...
18     ai robotics bigdata genomics stemcell advances...
19     applewatch to support both lte and nonlte mode...
28     beijing transit contactless mpayment system ex...
38     tweaktown pr asrockinfo introduces the x iot r...
41     pr introduces the x iot router for smart homes...
59     since its ipo home depot is actually outperfor...
64     amzns same day pick up locations are being cal...
68     would be amazed if jana partners manage to sel...
82           gs aapl amzn need to lead us higher spx dji
84     active traders try one of these free trading g...
85     xplr join us for play by play action on stocks...
86     amzn pzza restaurants are in a tech race to ma...
96     amzn part bmark offering guidance y y y y y y ...
102    hot options alert midday tuesday august bac dk...
108    there is a chance apple 

### Negative sentiment

In [29]:
df[(df['positive'] ==0) & (df['negative']  > 0)]['text'].head(20)

25     goog neonazi group moves to dark web after web...
43     will advances in ai ml robotics nanotech genom...
47     hpc ai ml bigdata may soon enable genome editi...
56     discussing the retail landscape department sto...
67     sitrep risk on mrk ceo youre fired amzn gs leg...
79     amzn aap wmt amazon will probably go onto crus...
80     dont worry about how many shares you can buy c...
95     amazon will probably go onto crush auto parts ...
119    tsla sa another risk factor for tesla shorts d...
145    microsoft acquires cloudcomputing orchestratio...
185    international business machines ibm fall to no...
187             the blue cloud collapses i told u ibm so
191    so u wont ask ginni about her two ibm failed a...
192    seekingalpha ibm watson disappointment risks f...
193    ibm watson disappointment risks further downwa...
200    china big market thus saith ginni so far zero ...
205    is ibms dividend yield killing strategic imper...
206    is ibms dividend yield k

In [30]:
print "Total tweets:", len(df)
print "Total tweets positive:",len(df[(df['positive'] >0) & (df['negative']  == 0)])
print "Total tweets negative:",len(df[(df['positive'] == 0) & (df['negative']  > 0)])
print "Tweets with no info:", len(df[(df['positive'] == 0) & (df['negative']  == 0)])
print "neutral tweets:", len(df[(df['positive'] >0) & (df['negative']  > 0)])

Total tweets: 8434
Total tweets positive: 2219
Total tweets negative: 1246
Tweets with no info: 4320
neutral tweets: 649


### Word2Vec

In [31]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('/Users/jenchieh/my_Github/GoogleNews-vectors-negative300.bin', binary=True)

### Converting each tweet into a vector

In [66]:
from gensim import matutils
matrix = []

df = df.reset_index().drop('index', axis=1)
        
for text in df.text:
    filtered_text = [model[w] for w in text.split() if w in model]
    if len(filtered_text):
        matrix.append(matutils.unitvec(np.array(filtered_text).mean(axis=0)))
    else:
        matrix.append(np.zeros(300))

In [67]:
matrix = np.array(matrix)
print matrix.shape
sim = np.dot(matrix, matrix.transpose())
print sim

(8434, 300)
[[ 1.          0.6802576   0.63090704 ...,  0.3900484   0.55352346
   0.12610915]
 [ 0.6802576   1.          0.64229834 ...,  0.32421411  0.46891389
   0.14354429]
 [ 0.63090704  0.64229834  1.         ...,  0.32880703  0.47104502
   0.24152685]
 ..., 
 [ 0.3900484   0.32421411  0.32880703 ...,  1.          0.59902306
   0.69015347]
 [ 0.55352346  0.46891389  0.47104502 ...,  0.59902306  1.          0.38061422]
 [ 0.12610915  0.14354429  0.24152685 ...,  0.69015347  0.38061422  1.        ]]


In [68]:
print sim.shape
np.fill_diagonal(sim, 0)
simdf = pd.DataFrame(list(sim[np.triu_indices(sim.shape[1], 1)]))
simdf.describe()

(8434, 8434)


,0
count,3.556196e+07
mean,4.484151e-01
std,1.223216e-01
min,-1.224153e-01
25%,3.672356e-01
50%,4.495172e-01
75%,5.318481e-01
max,1.000000e+00


In [69]:
#### Get the most similar tweets for each sentiment
pos = 19
most_similar = np.argmax(sim[pos][:])
print "similarity:", sim[pos][most_similar], most_similar
print re_df.iloc[pos].text
print re_df.iloc[most_similar].text

similarity: 0.85482778284 2468
with exascale expect quantum leap in ai ml robotics aerospace auto pharma goog msft intc aapl amd amzn f gm
quantum computing t revolutionize ai ml bigdata genomics pharma biotech robotics msft goog ibm


In [70]:
#### Get the most similar tweets for each sentiment
neg = 191
most_similar = np.argmax(sim[neg][:])
print "similarity:", sim[neg][most_similar], most_similar
print re_df.iloc[neg].text
print re_df.iloc[most_similar].text

similarity: 0.814871472026 3628
first trust advisors lp raises stake in box inc box
federated investors inc pa has million stake in box inc box


In [71]:
matrix.shape, df.shape

((8434, 300), (8434, 12))

### Adaboost Classifier

 - Train-Test Split
 - Grid Search on 3-fold CV
 - Perform predition by the best estimator

In [75]:
selected_idx = df.label != 0
y = df.label[selected_idx]
X = matrix[selected_idx]

y.shape, X.shape

((3465,), (3465, 300))

In [76]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, accuracy_score, make_scorer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [77]:
params={'n_estimators': [30, 100, 300]}

adb_cv = GridSearchCV(AdaBoostClassifier(), 
                      param_grid=params, scoring=make_scorer(f1_score),
                      cv=3, n_jobs=-1, verbose=2)

adb_cv.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] n_estimators=30 .................................................
[CV] n_estimators=30 .................................................
[CV] n_estimators=100 ................................................
[CV] n_estimators=30 .................................................
[CV] .................................. n_estimators=30, total=   2.5s
[CV] .................................. n_estimators=30, total=   2.5s
[CV] .................................. n_estimators=30, total=   2.6s
[CV] n_estimators=100 ................................................
[CV] n_estimators=100 ................................................
[CV] n_estimators=300 ................................................
[CV] ................................. n_estimators=100, total=   8.8s
[CV] n_estimators=300 ................................................
[CV] ................................. n_estimators=100, total=   8.9s
[CV] n_estimators=

[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:   26.5s remaining:    7.6s


[CV] ................................. n_estimators=300, total=  21.6s
[CV] ................................. n_estimators=300, total=  20.9s


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:   32.7s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [30, 100, 300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(f1_score), verbose=2)

In [78]:
estimator = adb_cv.best_estimator_
y_pred = estimator.predict(X_test)

print 'Accuracy:', accuracy_score(y_test, y_pred)
print 'F1 Score:', f1_score(y_test, y_pred)

Accuracy: 0.852813852814
F1 Score: 0.885650224215
